In [ ]:
import pandas as pd
import gzip
import json
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from math import cos, sin, pi
from bs4 import BeautifulSoup

In [ ]:
# DataFrame
def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')
def print_json_keys(filename):
    with gzip.open(filename, 'rb') as f:
        json_obj = json.loads(f.readline())
        print(json_obj.keys())
print_json_keys('meta_Appliances.json.gz')


In [ ]:
def get_product_at_index(path, product_index):
    for i, product_data in enumerate(parse(path)):
        if i == product_index:
            return product_data
    return None

In [ ]:
# Index of the product to display
product_index = 31
product_data = get_product_at_index('meta_Appliances.json.gz', product_index)

In [ ]:
# All data for a given index
keys_to_visualize = ['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'tech2', 
                     'brand', 'feature', 'rank', 'also_view', 'details', 'main_cat', 'similar_item']

def create_plotly_table(data, title):
    fig = go.Figure(data=[go.Table(
        header=dict(values=[title]),
        cells=dict(values=[data if isinstance(data, list) else [data]])
    )])
    fig.show()

for key in keys_to_visualize:
    data_for_key = product_data.get(key, 'N/A')  # Get the data for the key or 'N/A' if not found
    create_plotly_table(data_for_key, key)

In [ ]:
# Retrieve similar products
also_buy_asins = product_data.get('also_buy', []) if product_data else []


similar_items_titles = []
for asin in also_buy_asins:
    product_info = next((data for data in parse('meta_Appliances.json.gz') if data.get('asin') == asin), None)
    if product_info:
        similar_items_titles.append(product_info.get('title', 'Title not found'))

In [ ]:
# table of similar products
product_title = product_data.get('title', 'Product Title')
table_title = f"Similar items to '{product_title}'"

fig = go.Figure(data=[go.Table(
    header=dict(values=[table_title]),
    cells=dict(values=[similar_items_titles])
)]).update_layout(
    title=table_title,
    title_x=0.5
)

fig.show()

In [ ]:
# Function to calculate node positions
def calculate_node_positions(num_nodes, radius):
    angles = [2 * pi * i / num_nodes for i in range(num_nodes)]
    return [(radius * cos(angle), radius * sin(angle)) for angle in angles]

# Function to create edge traces
def create_edge_traces(node_positions):
    edges_x, edges_y = [], []
    for x, y in node_positions:
        edges_x.extend([0, x, None])
        edges_y.extend([0, y, None])
    return go.Scatter(x=edges_x, y=edges_y, line=dict(width=1, color='rgba(211,211,211,0.5)'),
                      hoverinfo='none', mode='lines', showlegend=False)

# Create node positions
num_related = len(also_buy_titles)
node_positions = calculate_node_positions(num_related, radius=5)

# Create edge traces
edge_trace = create_edge_traces(node_positions)

# Nodes and their ASIN labels
nodes_x, nodes_y = zip(*([0, 0] + list(zip(*node_positions))))
node_asin = ['Main'] + list(also_buy_titles.keys())

# Create node traces
node_trace = go.Scatter(
    x=nodes_x, y=nodes_y,
    mode='markers+text',
    text=node_asin,
    textposition="bottom center",
    hoverinfo='text',
    hovertext=[f"{product_data.get('title', 'Main Product')}"] + [title for title in also_buy_titles.values()],
    marker=dict(color=['lightskyblue'] + ['lavender'] * num_related, size=[20] + [10] * num_related),
    showlegend=False
)

fig = go.Figure(data=[edge_trace, node_trace])

# legend entries
main_product_title = product_data.get('title', 'Main Product Title')
fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers',
                         marker=dict(size=20, color='lightskyblue'),
                         name=f"Main Product: {main_product_title}", showlegend=True))
for asin, title in also_buy_titles.items():
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers',
                             marker=dict(size=10, color='lavender'),
                             name=f"{title}", showlegend=True))

# Update layout
fig.update_layout(
    legend_title_text=f'Similar items to "{main_product_title}"',
    legend=dict(yanchor="bottom", y=-0.5, xanchor="center", x=0.5),
    hovermode='closest',
    margin=dict(b=0, l=0, r=0, t=0, pad=4),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False, scaleanchor='y', scaleratio=1),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    plot_bgcolor='white'
)

# Show the figure
fig.show()
